# Notebook 1: Tokenization for LLMs

Welcome to the first notebook in our decoder-only transformer LLM course! In this notebook, we'll explore **tokenization** - the critical first step in processing text for language models.

## 📚 Learning Objectives

By the end of this notebook, you will:
1. Understand why tokenization is important for LLMs
2. Learn the difference between character-level, word-level, and subword tokenization
3. Implement Byte Pair Encoding (BPE) from scratch
4. Train a tokenizer on sample text
5. Use the Hugging Face tokenizers library
6. Handle special tokens (BOS, EOS, PAD, UNK)
7. Save and load tokenizers for reuse

## 🎯 Why Tokenization?

Language models don't process raw text - they work with **numerical representations**. Tokenization is the process of:
1. Breaking text into smaller units (tokens)
2. Mapping each token to a unique integer ID
3. Building a vocabulary of all possible tokens

**Key Trade-offs:**
- **Character-level**: Small vocabulary, long sequences (inefficient)
- **Word-level**: Large vocabulary (out-of-vocabulary problems)
- **Subword-level**: Best of both worlds! (BPE, WordPiece, Unigram)


In [ ]:
# Sample text for demonstration
sample_text = "The quick brown fox jumps over the lazy dog. It's a beautiful day!"

print("Original Text:")
print(sample_text)
print("\n" + "="*70)

# 1. Character-level tokenization
char_tokens = list(sample_text)
print(f"\n1️⃣ CHARACTER-LEVEL ({len(char_tokens)} tokens):")
print(char_tokens[:30], "...")
print(f"   Vocabulary size: {len(set(char_tokens))}")

# 2. Word-level tokenization (naive split)
word_tokens = sample_text.split()
print(f"\n2️⃣ WORD-LEVEL ({len(word_tokens)} tokens):")
print(word_tokens)
print(f"   Vocabulary size: {len(set(word_tokens))}")

# 3. Simple subword tokenization (preview)
# We'll implement this properly in the next section
print(f"\n3️⃣ SUBWORD-LEVEL (BPE - coming up next!):")
print("   Example: 'jumping' → ['jump', 'ing']")
print("   Balance between vocab size and sequence length")

In [ ]:
class SimpleBPETokenizer:
    """
    A simple implementation of Byte Pair Encoding (BPE) tokenizer.
    Educational implementation - not optimized for production use.
    """
    
    def __init__(self, vocab_size: int = 256):
        self.vocab_size = vocab_size
        self.vocab = {}  # token -> id
        self.inverse_vocab = {}  # id -> token
        self.merges = {}  # (token1, token2) -> merged_token
        self.bpe_ranks = {}  # merged_token -> rank
        
    def train(self, texts: List[str], verbose: bool = True):
        """
        Train the BPE tokenizer on a list of texts.
        
        Args:
            texts: List of text strings to train on
            verbose: Print progress information
        """
        if verbose:
            print("🔧 Training BPE Tokenizer...")
            print(f"Target vocabulary size: {self.vocab_size}")
        
        # Step 1: Initialize with character-level vocabulary
        # Start by splitting text into words and then characters
        word_freqs = defaultdict(int)
        for text in texts:
            words = text.split()
            for word in words:
                word_freqs[word] += 1
        
        # Convert words to character sequences with end-of-word marker
        splits = {}
        for word, freq in word_freqs.items():
            splits[word] = list(word) + ['</w>']
        
        # Build initial character vocabulary
        vocab = set()
        for word in splits.values():
            vocab.update(word)
        
        # Initialize vocab dict
        self.vocab = {char: idx for idx, char in enumerate(sorted(vocab))}
        self.inverse_vocab = {idx: char for char, idx in self.vocab.items()}
        
        if verbose:
            print(f"Initial vocabulary size: {len(self.vocab)}")
            print(f"Sample characters: {list(self.vocab.keys())[:20]}")
        
        # Step 2: Iteratively merge most frequent pairs
        num_merges = self.vocab_size - len(self.vocab)
        
        for i in range(num_merges):
            # Count all pairs
            pairs = defaultdict(int)
            for word, word_split in splits.items():
                for j in range(len(word_split) - 1):
                    pairs[(word_split[j], word_split[j + 1])] += word_freqs[word]
            
            if not pairs:
                break
            
            # Find most frequent pair
            best_pair = max(pairs, key=pairs.get)
            
            # Merge the pair in all words
            new_token = ''.join(best_pair)
            self.merges[best_pair] = new_token
            self.bpe_ranks[new_token] = i
            
            # Update splits
            new_splits = {}
            for word, word_split in splits.items():
                new_split = []
                j = 0
                while j < len(word_split):
                    if j < len(word_split) - 1 and \
                       (word_split[j], word_split[j + 1]) == best_pair:
                        new_split.append(new_token)
                        j += 2
                    else:
                        new_split.append(word_split[j])
                        j += 1
                new_splits[word] = new_split
            splits = new_splits
            
            # Add new token to vocabulary
            self.vocab[new_token] = len(self.vocab)
            self.inverse_vocab[len(self.inverse_vocab)] = new_token
            
            if verbose and (i + 1) % 50 == 0:
                print(f"  Merge {i+1}/{num_merges}: {best_pair[0]} + {best_pair[1]} → {new_token}")
        
        if verbose:
            print(f"✅ Training complete! Final vocabulary size: {len(self.vocab)}")
    
    def encode(self, text: str) -> List[int]:
        """Encode text to token IDs."""
        words = text.split()
        tokens = []
        
        for word in words:
            # Split word into characters
            word_tokens = list(word) + ['</w>']
            
            # Apply learned merges
            while len(word_tokens) > 1:
                # Find pairs that can be merged
                pairs = [(word_tokens[i], word_tokens[i + 1]) 
                        for i in range(len(word_tokens) - 1)]
                
                # Find the pair with the lowest rank (earliest merge)
                valid_pairs = [(pair, self.bpe_ranks.get(self.merges.get(pair, ''), float('inf')))
                              for pair in pairs if pair in self.merges]
                
                if not valid_pairs:
                    break
                
                # Merge the best pair
                best_pair = min(valid_pairs, key=lambda x: x[1])[0]
                merged = self.merges[best_pair]
                
                # Apply merge
                new_tokens = []
                i = 0
                while i < len(word_tokens):
                    if i < len(word_tokens) - 1 and \
                       (word_tokens[i], word_tokens[i + 1]) == best_pair:
                        new_tokens.append(merged)
                        i += 2
                    else:
                        new_tokens.append(word_tokens[i])
                        i += 1
                word_tokens = new_tokens
            
            # Convert to IDs
            for token in word_tokens:
                if token in self.vocab:
                    tokens.append(self.vocab[token])
                else:
                    # Handle unknown tokens (shouldn't happen in training data)
                    tokens.append(self.vocab.get('<UNK>', 0))
        
        return tokens
    
    def decode(self, token_ids: List[int]) -> str:
        """Decode token IDs back to text."""
        tokens = [self.inverse_vocab[tid] for tid in token_ids if tid in self.inverse_vocab]
        text = ''.join(tokens).replace('</w>', ' ')
        return text.strip()

# Create an instance
tokenizer = SimpleBPETokenizer(vocab_size=300)
print("✅ SimpleBPETokenizer class defined!")

## 🎯 Summary & Next Steps

### What We Learned:
✅ Different tokenization approaches (character, word, subword)  
✅ Byte Pair Encoding (BPE) algorithm  
✅ Implemented BPE from scratch  
✅ Used Hugging Face tokenizers for production  
✅ Special tokens (PAD, UNK, BOS, EOS)  
✅ Saving and loading tokenizers  

### Key Takeaways:
- **Subword tokenization** (BPE) balances vocabulary size and sequence length
- **Special tokens** are essential for model training
- **Hugging Face tokenizers** provide fast, production-ready implementations

### Next Notebook: `02_embeddings.ipynb`
In the next notebook, we'll convert token IDs into dense vector representations using **embedding layers** and **positional encodings**!

---

### 📝 Exercises (Optional):

1. **Train a larger tokenizer**: Use a real dataset (e.g., load text from a file)
2. **Compare vocab sizes**: Train tokenizers with different vocabulary sizes (100, 500, 5000) and compare
3. **Visualize token frequencies**: Create a histogram of token frequencies
4. **Domain-specific tokenizer**: Train a tokenizer on biomedical or chemical text
5. **Token length analysis**: Analyze average token length vs vocabulary size

In [ ]:
# Create directory for saving tokenizers
tokenizer_dir = "../data/tokenizers"
os.makedirs(tokenizer_dir, exist_ok=True)

# Train on a larger corpus (you would load your actual dataset here)
sample_corpus = training_corpus * 100  # Repeat for demo

# Train the HF tokenizer
print("Training Hugging Face tokenizer...")
hf_tokenizer.train_from_iterator(sample_corpus, trainer=trainer)

# Save the tokenizer
tokenizer_path = os.path.join(tokenizer_dir, "bpe_tokenizer.json")
hf_tokenizer.save(tokenizer_path)
print(f"✅ Tokenizer saved to: {tokenizer_path}")

# Load the tokenizer
loaded_tokenizer = Tokenizer.from_file(tokenizer_path)
print("✅ Tokenizer loaded successfully!")

# Test it
test_text = "The transformer architecture revolutionized NLP!"
encoding = loaded_tokenizer.encode(test_text)
print(f"\nTest: '{test_text}'")
print(f"Token IDs: {encoding.ids}")
print(f"Tokens: {encoding.tokens}")

## Part 5: Saving and Loading Tokenizers

For our LLM project, we'll need to save trained tokenizers and reuse them.

In [ ]:
# Install if needed (uncomment)
# !pip install tokenizers

from tokenizers import Tokenizer, models, trainers, pre_tokenizers, decoders, processors

def create_bpe_tokenizer(vocab_size=1000):
    """Create a BPE tokenizer using Hugging Face tokenizers library."""
    
    # Initialize a tokenizer with BPE model
    tokenizer = Tokenizer(models.BPE(unk_token="<UNK>"))
    
    # Set up pre-tokenization (how to split text before applying BPE)
    tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
    
    # Set up decoder
    tokenizer.decoder = decoders.BPEDecoder()
    
    # Create trainer
    trainer = trainers.BpeTrainer(
        vocab_size=vocab_size,
        special_tokens=["<PAD>", "<UNK>", "<BOS>", "<EOS>"],
        show_progress=True
    )
    
    return tokenizer, trainer

# Create tokenizer
hf_tokenizer, trainer = create_bpe_tokenizer(vocab_size=500)

print("✅ Hugging Face tokenizer created!")
print("\nSpecial tokens:")
print("  <PAD>: Padding token")
print("  <UNK>: Unknown token")  
print("  <BOS>: Beginning of sequence")
print("  <EOS>: End of sequence")

## Part 4: Using Hugging Face Tokenizers

For production use, we'll use the Hugging Face `tokenizers` library, which is much faster and more robust.

In [ ]:
# Sample training corpus
training_corpus = [
    "The quick brown fox jumps over the lazy dog",
    "A journey of a thousand miles begins with a single step",
    "To be or not to be that is the question",
    "All that glitters is not gold",
    "Better late than never",
    "The early bird catches the worm",
    "Actions speak louder than words",
]

# Train the tokenizer
tokenizer.train(training_corpus, verbose=True)

# Test encoding and decoding
test_text = "The quick brown fox"
encoded = tokenizer.encode(test_text)
decoded = tokenizer.decode(encoded)

print(f"\n{'='*70}")
print("🧪 Test Encoding/Decoding")
print(f"{'='*70}")
print(f"Original:  {test_text}")
print(f"Encoded:   {encoded}")
print(f"Decoded:   {decoded}")
print(f"Match:     {test_text.lower() == decoded.lower()}")

## Part 3: Training the BPE Tokenizer

Let's train our tokenizer on some sample text!

## Part 2: Byte Pair Encoding (BPE) from Scratch

**Byte Pair Encoding (BPE)** is a subword tokenization algorithm that:
1. Starts with a character-level vocabulary
2. Iteratively merges the most frequent pair of consecutive tokens
3. Stops when reaching a desired vocabulary size

This creates a vocabulary that balances coverage and efficiency!

### Algorithm Steps:
1. Initialize vocabulary with all characters
2. Count all adjacent token pairs
3. Merge the most frequent pair
4. Repeat until vocabulary size is reached

## Part 1: Understanding Different Tokenization Approaches

Let's start with a simple example to understand the three main approaches.

In [ ]:
# Import required libraries
import os
import sys
from collections import Counter, defaultdict
from typing import List, Dict, Tuple
import json
import pickle

# Add project root to path
sys.path.append('..')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print(f"Python version: {sys.version}")